In [177]:
import pandas as pd
import numpy as np
import re


# import data

In [178]:
example = pd.read_csv('example.txt',header=None, sep=',', engine='python')
example['x2y1'] = example[1].apply(lambda x: re.split('\s+->\s+',x))
example['x1'] = example[0]
example['y1'] = example['x2y1'].apply(lambda x: int(x[0]))
example['x2'] = example['x2y1'].apply(lambda x: int(x[1]))
example['y2'] = example[2]
example.drop([0,1,2,'x2y1'], axis=1, inplace=True)
example.head()

,x1,y1,x2,y2
0,0,9,5,9
1,8,0,0,8
2,9,4,3,4
3,2,2,2,1
4,7,0,7,4


In [179]:
original_input = pd.read_csv('input.txt',header=None, sep=',', engine='python')
original_input['x2y1'] = original_input[1].apply(lambda x: re.split('\s+->\s+',x))
original_input['x1'] = original_input[0]
original_input['y1'] = original_input['x2y1'].apply(lambda x: int(x[0]))
original_input['x2'] = original_input['x2y1'].apply(lambda x: int(x[1]))
original_input['y2'] = original_input[2]
original_input.drop([0,1,2,'x2y1'], axis=1, inplace=True)
original_input.head()

,x1,y1,x2,y2
0,88,177,566,655
1,346,264,872,264
2,409,631,506,534
3,300,216,300,507
4,80,370,193,483


In [199]:
# line_segments = example.copy().astype(int)
line_segments = original_input.copy().astype(int)#example.copy()
line_segments.head()

,x1,y1,x2,y2
0,88,177,566,655
1,346,264,872,264
2,409,631,506,534
3,300,216,300,507
4,80,370,193,483


In [200]:
xmin = line_segments[['x1', 'x2']].min().min()
xmax = line_segments[['x1', 'x2']].max().max()
ymin = line_segments[['y1', 'y2']].min().min()
ymax = line_segments[['y1', 'y2']].max().max()
((xmin, xmax),(ymin, ymax))

((10, 987), (11, 989))

In [201]:
initial_board = np.zeros((xmax+1, ymax+1)).astype(int)
initial_board

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Find number of lines crossing each point
loop over all lines and increase counter in board of the points the line crosses

In [202]:
board_w_counters = initial_board.copy()
for idx, row in line_segments.iterrows():
    x1 = min(row.x1, row.x2)
    x2 = max(row.x1, row.x2)
    y1 = min(row.y1, row.y2)
    y2 = max(row.y1, row.y2)
    if (row.x1 == row.x2) or (row.y1 == row.y2):
        board_w_counters[x1:x2+1, y1:y2+1] = board_w_counters[x1:x2+1, y1:y2+1] + 1
(board_w_counters >= 2).sum(axis=None)

5835

In [203]:
board_w_counters

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Part B

In [204]:
# create list of indices from diagonal
x1=1
x2=3
y1=2
y2=4
# points that it crosses: (1,2), (2,3), (3,4)
points = np.array([(int(x1+p_i), int(y1+p_i)) for p_i in range(0,x2-x1+1)])
test_board = board_w_counters.copy()[:5,:5]
print(points)

[[1 2]
 [2 3]
 [3 4]]


In [205]:
board_w_counters = initial_board.copy()
for idx, row in line_segments.iterrows():
    x1 = min(row.x1, row.x2)
    x2 = max(row.x1, row.x2)
    y1 = min(row.y1, row.y2)
    y2 = max(row.y1, row.y2)
    if (row.x1 == row.x2) or (row.y1 == row.y2):
        board_w_counters[x1:x2+1, y1:y2+1] += 1#board_w_counters[x1:x2+1, y1:y2+1] + 1
    elif y2-y1 == x2-x1:
        indices = [(x1+p_i, y1+p_i) for p_i in range(0,x2-x1+1)]
        if row.x2 > row.x1:
            x_inc = 1
        else:
            x_inc = -1
        if row.y2 > row.y1:
            y_inc = 1
        else:
            y_inc = -1
        points = np.array([(int(row.x1+x_inc*p_i), int(row.y1+y_inc*p_i)) for p_i in range(0,x2-x1+1)])
        for point_x, point_y in points:
            board_w_counters[point_x, point_y] += 1
    else:
        print((row, ))
        
(board_w_counters >= 2).sum(axis=None)

17013

In [198]:
board_w_counters.transpose()

array([[1, 0, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 2, 0, 0],
       [0, 0, 2, 0, 1, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 2, 0, 2, 0, 0],
       [0, 1, 1, 2, 3, 1, 3, 2, 1, 1],
       [0, 0, 0, 1, 0, 2, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [2, 2, 2, 1, 1, 1, 0, 0, 0, 0]])